# Stencil Patterns with Shared Memory

This notebook focuses on implementing stencil patterns, a crucial technique for parallel processing on GPUs.
We'll explain these concepts through concrete code examples, combining CuPy's RawModule with CUDA kernels.

In this notebook, readers will learn about the fundamental ideas behind stencil patterns and their efficient GPU implementation in a step-by-step manner.
To illustrate this, we've chosen the simplest example: a one-dimensional array's moving average.
We'll specifically explain Shared Memory, which significantly impacts GPU computation performance.

First, we'll demonstrate an implementation where threads within a block use shared memory to share data and collaborate on computations.
Subsequently, we'll show an implementation that achieves cooperative processing without Shared Memory, by making the Cooperative Thread Array (CTA) subdivision unit a Warp.
This involves data exchange using shuffle instructions. The latter approach allows inter-thread communication without the constraints of Shared Memory.
Through these implementations, the concepts of cooperation at both the Warp and block levels will be understood.
This will lay the groundwork for utilizing NVIDIA's CUDA universal parallel primitives library, CUB, which we'll cover next time.

First, we define `block_size`, which specifies the number of threads in a GPU thread block, and `length`, the total number of elements in the array to be processed.

`block_size` determines how many threads constitute a GPU thread block.
Threads launched with this `block_size` will cooperate using Shared Memory on the same Streaming Multiprocessor and synchronize with `__syncthreads()`.
Here, we've set it to the maximum value of `1024`.
`length` specifies the total number of elements in the data array to be processed.
In this example, since `length` is greater than `block_size`, a single thread block cannot process all data.
When launching multiple thread blocks (`2` blocks in this case) to process data in a distributed manner, kernel code needs to account for "block boundaries."
Let's examine how this is handled in the following CUDA kernel code.

In [ ]:
import os
import math
import numpy as np
import cupy as cp

err_eps = 1E-7
block_size = 1024
length = 2048

dn = os.path.join(os.getcwd(), 'kernels')
fpfn = os.path.join(dn, '03_stencil_patterns_1.cu')
with open(fpfn, 'r') as f:
  cuda_source = f.read()
print(cuda_source)

extern "C" __global__ void movingAverage2(float *y,
  const float *__restrict__ x)
{
  const int index = blockIdx.x * blockDim.x + threadIdx.x;
  if (index >= STENCIL_PATTERNS_1_LENGTH)
  {
    return;
  }

  __shared__ float xShared[STENCIL_PATTERNS_1_BLOCK];
  xShared[threadIdx.x] = x[index];
  __syncthreads();

  if (threadIdx.x == (STENCIL_PATTERNS_1_BLOCK - 1))
  {
    y[index] = (xShared[threadIdx.x] + x[(index + 1) % STENCIL_PATTERNS_1_LENGTH])/2;
  }
  else
  {
    y[index] = (xShared[threadIdx.x] + xShared[threadIdx.x + 1])/2;
  }
}



The `movingAverage2` CUDA kernel implements a stencil pattern for calculating the moving average of a one-dimensional array using Shared Memory.

First, `xShared`, declared with the `__shared__` memory space specifier, becomes Shared Memory.
Each thread copies data from global memory to Shared Memory using `xShared[threadIdx.x] = x[index];`.
It's crucial to note that while `index` is the absolute global memory index, `threadIdx.x` is the relative index within the block for Shared Memory.
Since Shared Memory is shared within a block, global memory indices should not be used directly.
`__syncthreads();` ensures that all threads within the block have completed writing to Shared Memory.

Next, the moving average is calculated by reading adjacent elements from Shared Memory.
When reading these adjacent elements, two types of boundary handling must be considered: array length boundaries and block boundaries.
Particular care is needed for block boundaries; only the thread with the largest `threadIdx.x` directly reads data from global memory again, after considering array length boundaries.

Shared Memory in CUDA is a very fast on-chip memory region on the GPU. It offers significantly higher bandwidth and lower latency compared to global memory and is used for data sharing among threads within the same thread block. To achieve this high speed, Shared Memory is divided into multiple "banks," allowing for high parallelism by accessing multiple banks simultaneously. However, if different threads attempt to access the same bank concurrently, a "bank conflict" occurs, serializing access and degrading performance.

In [ ]:
cuda_source = cuda_source.replace('STENCIL_PATTERNS_1_BLOCK', str(block_size))
cuda_source = cuda_source.replace('STENCIL_PATTERNS_1_LENGTH', str(length))
module = cp.RawModule(code=cuda_source)
module.compile()

After embedding constants via string replacement, we compile the `RawModule` and upload arrays to the GPU.

In [ ]:
x = np.arange(0, length, dtype=np.float32)
x_gpu = cp.array(x, dtype=cp.float32)
y_gpu = cp.empty_like(x)
assert x_gpu.flags.c_contiguous
assert y_gpu.flags.c_contiguous

We retrieve the CUDA kernel, specify the block and grid sizes, and launch the kernel.

In [ ]:

gpu_func = module.get_function('movingAverage2')
sz_block = block_size,
sz_grid = math.ceil(length / sz_block[0]),
gpu_func(
  block=sz_block, grid=sz_grid,
  args=(y_gpu, x_gpu)
)
cp.cuda.runtime.deviceSynchronize()

We calculate the correct result using `numpy.roll` for cyclic boundary handling and compare the results.
If the computation is successful, no assertion error will occur.

In [ ]:
y = y_gpu.get()
y_ref = (x + np.roll(x, -1))/2

err = np.abs(y_ref - y)
assert np.max(err) < err_eps

Next, we'll implement a stencil pattern for calculating the moving average of a one-dimensional array using Warp-level cooperative processing.
This approach focuses on thread collaboration within a Warp.

In [ ]:
fpfn = os.path.join(dn, '03_stencil_patterns_2.cu')
with open(fpfn, 'r') as f:
  cuda_source = f.read()
print(cuda_source)

#include <cooperative_groups.h>
namespace cg = cooperative_groups;

extern "C" __global__ void movingAverage2CTAWarp(float *y,
  const float *__restrict__ x)
{
	auto cta = cg::this_thread_block();
	const int ctaIndex = cta.thread_rank();
  if (ctaIndex >= STENCIL_PATTERNS_2_LENGTH)
  {
    return;
  }
  auto tile = cg::tiled_partition<STENCIL_PATTERNS_2_WARP_SIZE>(cta);
  float val = x[ctaIndex];
  float average = (tile.shfl_down(val, 1) + val) / 2;
  tile.sync();
  if (tile.thread_rank() == STENCIL_PATTERNS_2_WARP_SIZE - 1)
  {
    average = (val + x[(ctaIndex + 1) % STENCIL_PATTERNS_2_LENGTH])/2;
  }
  cta.sync();
  y[ctaIndex] = average;
}



The `movingAverage2CTAWarp` kernel leverages Cooperative Groups library, a powerful CUDA feature.
In CUDA, a Warp is the fundamental unit by which the GPU executes and manages threads, typically consisting of 32 threads.
These threads operate under a SIMT (Single Instruction, Multiple Threads) architecture, executing the same instruction simultaneously.
Cooperative Groups, with `#include <cooperative_groups.h>`, enables flexible management of thread groups and fast data exchange (e.g., shuffle instructions) within a Warp.

To compile this kernel using CuPy's default NVRTC backend, just set the `enable_cooperative_groups=True` option when creating `cupy.RawModule`.

In [ ]:
warp_size = 32
cuda_source = cuda_source.replace('STENCIL_PATTERNS_2_LENGTH', str(length))
cuda_source = cuda_source.replace('STENCIL_PATTERNS_2_WARP_SIZE', str(warp_size))
module = cp.RawModule(code=cuda_source, enable_cooperative_groups=True)
module.compile()

We retrieve the newly compiled CUDA kernel from the `RawModule`, specify the block and grid sizes, and launch the kernel.

In [ ]:

gpu_func = module.get_function('movingAverage2CTAWarp')
sz_block = block_size,
sz_grid = math.ceil(length / sz_block[0]),
gpu_func(
  block=sz_block, grid=sz_grid,
  args=(y_gpu, x_gpu)
)
cp.cuda.runtime.deviceSynchronize()

If the computation is successful, no assertion error will occur.

In [ ]:
y = y_gpu.get()
y_ref = (x + np.roll(x, -1))/2

err = np.abs(y_ref - y)
assert np.max(err) < err_eps

In this notebook, we explained two main implementation methods for stencil patterns in CUDA, using the moving average as an example.

First, block-level cooperation using Shared Memory required explicit memory loads for data reuse and synchronization via `__syncthreads()`.
In contrast, Warp-level cooperation allowed for faster communication by directly exchanging data between threads within a Warp using shuffle instructions (like `__shfl_sync`). This approach bypasses Shared Memory for inter-thread communication.

A key takeaway from both methods was the need for specific considerations for their respective boundary handling (end of block and end of Warp). Additionally, while the Cooperative Groups features used for Warp-level cooperation can be compiled with CuPy's default NVRTC backend by setting `enable_cooperative_groups=True`, using the NVCC backend will become necessary in the next notebook when we integrate the CUB library.